In [30]:
from cobra.io import read_sbml_model 
from cobra import Model, Reaction, Metabolite

model = read_sbml_model('models/iMM904_progesterone.xml') #load model

model.medium

{'EX_fe2_e': 999999.0,
 'EX_glc__D_e': 10.0,
 'EX_h2o_e': 999999.0,
 'EX_h_e': 999999.0,
 'EX_k_e': 999999.0,
 'EX_na1_e': 999999.0,
 'EX_so4_e': 999999.0,
 'EX_nh4_e': 999999.0,
 'EX_o2_e': 2.0,
 'EX_pi_e': 999999.0}

Maximum theoretical productivity and yield of progesterone on different carbon sources:

In [32]:
medium = model.medium
with model: 
    solution = model.optimize()
    model.medium = medium
    model.objective = model.reactions.R02216 # objective is to lead flux through the reaction producing progesterone top maximise production
    progesterone_production = model.optimize().objective_value # define new model with maximum production of progesterone
    glc_flux = CITtcp
    print('Maximum theoretical productivity of progesterone: ', progesterone_production, 'mmol/gDW*h') # calculate productivity
    print('Maximum theoretical yield of progesterone on glucose: ', progesterone_production / (-1 * model.reactions.EX_glc__D_e.flux), 'mmol progesterone/mmol glucose') # calculate yield on glucose
    
    
    

SolverError: CPLEX Error  1016: Community Edition. Problem size limits exceeded. Purchase at http://ibm.biz/error1016.

As seen xxxx

In [ ]:

with model:
    
    print('Maximum theoretical yield of progesterone on galactose: ', progesterone_production / (-1 * model.reactions.EX_gal_e.flux), 'mmol progesterone/mmol galactose') # calculate yield on galactose
    print('Maximum theoretical yield of progesterone on fructose: ', progesterone_production / (-1 * model.reactions.EX_fru_c.flux), 'mmol progesterone/mmol fructose') # calculate yield on fructose


The xxxxxx